# Introduction to bokeh

## Bokeh server app
---

<img src='./images/logos.3.600.wide.png' height='250' width='300' style="float:right">

We briefly mentioned one of the major donwsides to the client-side version of Bokeh: limitations within the browser in terms of how much data can be handled at any given time.

Bokeh Server attempts to get past that enabling the browser and the data to be sync'ed more effectively:
 

* **Event response**: respond to UI and tool events generated in a browser with computations or queries using the full power of Python
* **Server-side updates**: automatically push server-side updates to the UI (i.e. widgets or plots in a browser)
* **Streaming updates** use periodic, timeout, and asynchronous callbacks to drive streaming updates

Several flavors of use for the Bokeh Server:

* local / individual use
* deployable application

Our focus will be on local and individual use. This should get you pointed in the right direction should you want to create a deployable application.





# Bokeh Server stock example

We will start off by standing up a Bokeh Server that produces visualizations focused on stock exchange data

* Prep your environment
* Download the data
* Generate the code
* Run the server


## Prep  your environment

### Linux/Mac
```bash
mkdir bserver
cd bserver
conda create -n bserver python=3
source activate bserver
conda install bokeh jupyter ipython pandas
```

### WIndows
```bash
mkdir bserver
cd bserver
conda create -n bserver python=3
activate bserver
conda install bokeh jupyter ipython pandas
```

## Download the data

1. Navigate to the Bokeh github site: https://github.com/bokeh/bokeh <img src='./images/github_download.png' height='1000' width='600' style='float:right'>
1. Click on the **Clone or Downland** button
1. Click on the **Download Zip** button 
1. Save the file to your `bserver` folder
1. Unzip the file, if necessary
1. Type the following on the command line to download the datafiles for the **STOCKS** application:
   1. `cd bokeh-master/examples/app/stocks/`
   1. `python download_sample_data.py`


## Create the code

In this case, we will be reviewing the code that is already present in the stocks folder. We will look at the code here in the notebook book, but to access it directly, you can open the `main.py` file from the stocks folder in your text editor.


### HEADER --------------------------------------------------
```
''' Create a simple stocks correlation dashboard.

Choose stocks to compare in the drop down widgets, and make selections
on the plots to update the summary and histograms accordingly.

.. note::
    Running this example requires downloading sample data. See
    the included `README`_ for more information.

Use the ``bokeh serve`` command to run the example by executing:

    bokeh serve stocks

at your command prompt. Then navigate to the URL

    http://localhost:5006/stocks

.. _README: https://github.com/bokeh/bokeh/blob/master/examples/app/stocks/README.md

'''
```


### IMPORTS -------------------------------------------------
```
try:
    from functools import lru_cache
except ImportError:
    # Python 2 does stdlib does not have lru_cache so let's just
    # create a dummy decorator to avoid crashing
    print ("WARNING: Cache for this example is available on Python 3 only.")
    def lru_cache():
        def dec(f):
            def _(*args, **kws):
                return f(*args, **kws)
            return _
        return dec

from os.path import dirname, join

import pandas as pd

from bokeh.io import curdoc
from bokeh.layouts import row, column
from bokeh.models import ColumnDataSource
from bokeh.models.widgets import PreText, Select
from bokeh.plotting import figure

```


### CREATE OR LOAD DATA/ENRICHMENTS -------------------------
```
DATA_DIR = join(dirname(__file__), 'daily')

DEFAULT_TICKERS = ['AAPL', 'GOOG', 'INTC', 'BRCM', 'YHOO']

def nix(val, lst):
    return [x for x in lst if x != val]

@lru_cache()
def load_ticker(ticker):
    fname = join(DATA_DIR, 'table_%s.csv' % ticker.lower())
    data = pd.read_csv(fname, header=None, parse_dates=['date'],
                       names=['date', 'foo', 'o', 'h', 'l', 'c', 'v'])
    data = data.set_index('date')
    return pd.DataFrame({ticker: data.c, ticker+'_returns': data.c.diff()})

@lru_cache()
def get_data(t1, t2):
    df1 = load_ticker(t1)
    df2 = load_ticker(t2)
    data = pd.concat([df1, df2], axis=1)
    data = data.dropna()
    data['t1'] = data[t1]
    data['t2'] = data[t2]
    data['t1_returns'] = data[t1+'_returns']
    data['t2_returns'] = data[t2+'_returns']
    return data

# set up widgets

stats = PreText(text='', width=500)
ticker1 = Select(value='AAPL', options=nix('GOOG', DEFAULT_TICKERS))
ticker2 = Select(value='GOOG', options=nix('AAPL', DEFAULT_TICKERS))

# set up plots

source = ColumnDataSource(data=dict(date=[], t1=[], t2=[], t1_returns=[], t2_returns=[]))
source_static = ColumnDataSource(data=dict(date=[], t1=[], t2=[], t1_returns=[], t2_returns=[]))
tools = 'pan,wheel_zoom,xbox_select,reset'

```


### GENERATE FIGURE -----------------------------------------
```
corr = figure(plot_width=350, plot_height=350,
              tools='pan,wheel_zoom,box_select,reset')
```
### GENERATE GLYPHS -----------------------------------------
```
corr.circle('t1_returns', 't2_returns', size=2, source=source,
            selection_color="orange", alpha=0.6, nonselection_alpha=0.1, selection_alpha=0.4)
```
### GENERATE FIGURE -----------------------------------------
```
ts1 = figure(plot_width=900, plot_height=200, tools=tools, x_axis_type='datetime', active_drag="xbox_select")
```
### GENERATE GLYPHS -----------------------------------------
```
ts1.line('date', 't1', source=source_static)
ts1.circle('date', 't1', size=1, source=source, color=None, selection_color="orange")
```
### GENERATE FIGURE -----------------------------------------
```
ts2 = figure(plot_width=900, plot_height=200, tools=tools, x_axis_type='datetime', active_drag="xbox_select")
ts2.x_range = ts1.x_range
```
### GENERATE GLYPHS -----------------------------------------
```
ts2.line('date', 't2', source=source_static)
ts2.circle('date', 't2', size=1, source=source, color=None, selection_color="orange")

```


### ADD ATTRIBUTES, ANNOTATIONS, INTERACTIONS ---------------
```
# set up callbacks

def ticker1_change(attrname, old, new):
    ticker2.options = nix(new, DEFAULT_TICKERS)
    update()

def ticker2_change(attrname, old, new):
    ticker1.options = nix(new, DEFAULT_TICKERS)
    update()

def update(selected=None):
    t1, t2 = ticker1.value, ticker2.value

    data = get_data(t1, t2)
    source.data = source.from_df(data[['t1', 't2', 't1_returns', 't2_returns']])
    source_static.data = source.data

    update_stats(data, t1, t2)

    corr.title.text = '%s returns vs. %s returns' % (t1, t2)
    ts1.title.text, ts2.title.text = t1, t2

def update_stats(data, t1, t2):
    stats.text = str(data[[t1, t2, t1+'_returns', t2+'_returns']].describe())

ticker1.on_change('value', ticker1_change)
ticker2.on_change('value', ticker2_change)

def selection_change(attrname, old, new):
    t1, t2 = ticker1.value, ticker2.value
    data = get_data(t1, t2)
    selected = source.selected['1d']['indices']
    if selected:
        data = data.iloc[selected, :]
    update_stats(data, t1, t2)

source.on_change('selected', selection_change)

# set up layout

widgets = column(ticker1, ticker2, stats)
main_row = row(corr, widgets)
series = column(ts1, ts2)
layout = column(main_row, series)

```

### CREATE OUTPUTS ------------------------------------------
```
# initialize

update()

curdoc().add_root(layout)
curdoc().title = "Stocks"

```

## Run the server

From **within** the **stocks** folder, execute the following command from the command line:

1. `bokeh serve --show .`

## Navigation
---

| Previous | Next |
|:----|-----:|
| <<< [History](history.ipynb) | [XXX](./xxx.ipynb) >>> |